In [1]:
import torch
import torch.nn as nn

In [2]:
"""
선형회귀(fully connected)
y = Wx + b
"""
tensor = torch.arange(4, dtype=torch.float32).view(2, 2)
linear = nn.Linear(2, 5)
print(f"before linear: {tensor.shape}")
print(f"after linear: {linear(tensor).shape}")

before linear: torch.Size([2, 2])
after linear: torch.Size([2, 5])


In [3]:
"""
단위행렬
"""
tensor = torch.arange(4, dtype=torch.float32).view(2, 2)
identity = nn.Identity()
print(torch.equal(tensor, identity(tensor)))

True


In [4]:
"""
모듈 컨테이너
파이썬 기본 컨테이너를 사용해도 동작은 하지만 모듈로 인식하지 않아 모듈의 메소드들을 사용할 수 없다
"""
class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.python_list = [
            nn.Linear(2, 2),
        ]
        self.pytorch_module_list = nn.ModuleList([
            nn.Linear(2, 2)
        ])
        self.pytorch_module_dict = nn.ModuleDict({
            "linear": nn.Linear(2, 2)
        })
        self.pytorch_module_sequential = nn.Sequential(
            nn.Linear(2, 2)
        )
    
    def forward(self, x):
        x = self.python_list[0](x)
        x = self.pytorch_module_list[0](x)
        x = self.pytorch_module_dict["linear"](x)
        x = self.pytorch_module_sequential(x)
        return x

model = Model()
print(model(torch.arange(4, dtype=torch.float).view(2, 2)))
print(model)

tensor([[0.7212, 0.1551],
        [0.7754, 0.2750]], grad_fn=<AddmmBackward0>)
Model(
  (pytorch_module_list): ModuleList(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
  (pytorch_module_dict): ModuleDict(
    (linear): Linear(in_features=2, out_features=2, bias=True)
  )
  (pytorch_module_sequential): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
)


In [5]:
"""
Tensor: 
    gradient 계산 X(default)
    값 업데이트 X
    모델 저장시 값 저장 X

Parameter: 
    gradient 계산 O
    값 업데이트 O
    모델 저장시 값 저장 O

Buffer
    gradient 계산 X
    값 업데이트 X
    모델 저장시 값 저장 O
"""

'\nTensor: \n    gradient 계산 X(default)\n    값 업데이트 X\n    모델 저장시 값 저장 X\n\nParameter: \n    gradient 계산 O\n    값 업데이트 O\n    모델 저장시 값 저장 O\n\nBuffer\n    gradient 계산 X\n    값 업데이트 X\n    모델 저장시 값 저장 O\n'

In [9]:
class SmallModel(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    def forward(self, x):
        return x + 1

class MiddleModel(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.small1 = SmallModel()
        self.small2 = SmallModel()
    
    def forward(self, x):
        x = self.small1(x)
        x = self.small2(x)
        return x

class LargeModel(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.middle1 = MiddleModel()
        self.middle2 = MiddleModel()
    
    def forward(self, x):
        x = self.middle1(x)
        x = self.middle2(x)
        return x

x = torch.tensor([0])
model = LargeModel()

In [10]:
for name, module in model.named_children():
    print(f"name: {name}\nmodule: \n{module}")

name: middle1
module: 
MiddleModel(
  (small1): SmallModel()
  (small2): SmallModel()
)
name: middle2
module: 
MiddleModel(
  (small1): SmallModel()
  (small2): SmallModel()
)


In [ ]:
"""
hook: 특정 코드를 실행 중간에 끼워넣는 방법

forward hook(모듈에만 사용 가능)
    forward_pre_hook: 순전파 전 실행
    forward_hook: 순전파때 실행

backward hook(모듈, 텐서에 사용 가능)
    full_backward_hook: 역전파때 실행(모듈)
    hook: 역전파때 실행(텐서)
"""

In [13]:
"""
apply: 모듈 전체에 적용되는 함수
적용되는 순서는 postorder traversal 순서(pytorch는 모델이 트리구조)

ex) .cuda()를 사용하면 모듈 전체에 .cuda()가 적용됨 
"""

@torch.no_grad() # 데코레이터
def init_weight(module):
    if type(module) == nn.Linear:
        module.weight.fill_(1.0)
        print(f"weight init: {module.weight}")

model = nn.Sequential(nn.Linear(2, 2), 
                      nn.Linear(2, 2))
model.apply(init_weight)

weight init: Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
weight init: Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [16]:
"""
apply로 extra_repr를 수정해 원하는 출력이 나오도록 변경하기
"""
from functools import partial

def new_repr(self):
    return f"module name: {self.__class__.__name__}"

def add_repr(module):
    module.extra_repr = partial(new_repr, module)

repr_model = model.apply(add_repr)
print(repr(repr_model))

Sequential(
  module name: Sequential
  (0): Linear(module name: Linear)
  (1): Linear(module name: Linear)
)
